<center><h1>Telco Churn Project</h1></center>

# Table of Contents <a name ='toc'></a>
1. [Project Summary](#project_summary)
    1. [Project Objectives](#project_objectives)
    2. [Business Goals](#business_goals)
    3. [Audience](#audience)
    4. [Deliverables](#deliverables)
    5. [Data Dictonary](#data_dict)
3. [Executive Summary](#exe_summ)
2. [Data Acqisition](#data_acquisition)
5. [Preparation](#preparation)
6. [Exploratory Data Analysis](#exp_data_analysis)
2. [Drivers of Churn](#drivers_of_churn)
3. [Machine Learning Construction](#ml_construction)
7. [Statistical Testing](#stat_testing)
8. [Modeling](#modeling)
9. [Model Evaluation](#model_eval)


<hr style="border-top: 10px groove tan; margin-top: 1px; margin-bottom: 1px"></hr>

## Project Summary <a name="project_summary"></a>
<hr style="border-top: 10px groove tan; margin-top: 1px; margin-bottom: 1px"></hr>

### Project Obectives <a name='project_objectives'></a>
> - Create a Jupyter Notebook Report that shows processes and analysis with the goal of finding drivers for customer churn.
> - Within README.md file, include project description with goals, inital hypotheses, a data dictonary, project planning, instructions on how to recreate your project, answers to hypotheses, key findings, recommendations, and takeaways from the project.
> - CSV file with customer_id, probabilty of churn, and prediction of churn (1 = churn, 0 = not_churn). These predictions will be dervied from the best performing model on the test portion of the data. 
> - Any abstracted modules that are created to make the presentation more clean, during the acquistion and preparation of data.
> - Notebook walkthrough and presentation with a high-level overview of the entire project.

### Business Goals <a name='business_goals'></a>
> - Find drivers for customer churn at Telco. Why are customers churning?
> - Construct a machine-learning classification model that accurately predicts customer churn.
> - Document your process well enough to be presented or read like a report.

### Audience <a name='audience'></a>
> - Target audience is the Codeup Data Science Team.


### Deliverables  <a name='deliverables'></a>
> - A final report within Juypter Notebooks
> - A final report presentation using Juypter Notebooks
> - Modules necessary to recreate project

### Data Dictionary <a name='data_dict'></a>
>-

<div style="text-align: right"><a href=#toc>Table of Contents</a></div>
<hr style="border-top: 10px groove tan; margin-top: 1px; margin-bottom: 1px"></hr>

